# Working with masks

## Introduction

Scipp supports non-destructive masks stored alongside data.
In this tutorial we learn how to create and use masks.

This tutorial contains exercises, but solutions are included directly.
We encourage you to download this notebook and run through it step by step before looking at the solutions.
We recommend to use a recent version of *JupyterLab*:
The solutions are included as hidden cells and shown only on demand.

As a side effect, the exercises will help in getting more familiar with the basic concepts of operations.

First, in addition to importing `scipp`, we import `scippneutron` since this is required for loading Nexus files:

In [ ]:
import scipp as sc
import scippneutron as scn
import numpy as np

We  start by loading some data, in this case measured with a prototype of the [LoKI](https://europeanspallationsource.se/instruments/loki) detectors at the [LARMOR beamline](https://www.isis.stfc.ac.uk/Pages/Larmor.aspx):

In [ ]:
dg = scn.data.tutorial_dense_data()
dg

In [ ]:
data = dg['detector']  # the actual measured counts
counts = data.sum('tof')  # used later
data

Note that the exercises in the following are fictional and do not represent the actual SANS data reduction workflow.

Masks are variables with `dtype=bool`, stored in the `masks` dict of a data array.
The result of comparison between variables can thus be used as masks:

In [ ]:
data.coords['spectrum'] < sc.scalar(100)

## Exercise 1: Masking a prompt pulse

1. Create a prompt-pulse mask for the region between $17500~\mathrm{\mu s}$ and $19000~\mathrm{\mu s}$.
   Notes:
   - Define, e.g., `prompt_start = 17500 * sc.Unit('us')` and the same for the upper bound of the prompt pulse.
   - Use comparison operators such as `==`, `<=` or `>`.
   - Combine multiple conditions into one using `&` ("and"), `|` ("or"), or `^` ("exclusive or").
   - Masks are stored in a data array by storing them in the `masks` dictionary, e.g., `data.masks['prompt-pulse'] = ...`.
   - If something goes wrong, masks can be removed with Python's `del`, e.g., `del data.masks['wrong']`.
   - If you run into an error regarding a length mismatch when inserting the coordinate, remember that `'tof'` is a bin-edge coordinate, i.e., it is by 1 longer than the number of bins.
     Use, e.g., only the left bin edges, i.e., all but the last, to create the masks.
     This can be achieved using slicing, e.g., `array[dim_name, start_index:end_index]`.
2. Use the HTML view and plot the data after masking to explore the effect.
3. Pass a `dict` containing `counts` (computed above as `counts = data.sum('tof')`) and the equivalent counts computed *after* masking to `sc.plot`.
   Use this to verify that the prompt-pulse mask results in removal of counts.

### Solution

In [ ]:
prompt_start = 17500 * sc.Unit('us')
prompt_stop = 19000 * sc.Unit('us')
tof = data.coords['tof']
mask = (tof['tof', :-1] > prompt_start) & (tof['tof', :-1] < prompt_stop)
data.masks['prompt-pulse'] = mask
data

In [ ]:
data.hist(spectrum=500).transpose().plot()

In [ ]:
tof = data.coords['tof']
sc.plot({'before': counts, 'after': data.sum('tof')})

## Exercise 2: Masking spatially

By masking an `x` range, mask the end of the tubes.

- Define `x = data.coords['position'].fields.x` to extract only the x-component of the position coordinate.
- Create the masks.
- Use the instrument view (`scn.instrument_view(data)`) to inspect the result.

### Solution

In [ ]:
x = data.coords['position'].fields.x
data.masks['tube-ends'] = sc.abs(x) > 0.5 * sc.Unit('m')
scn.instrument_view(sc.sum(data, 'tof'), norm='log')  # norm='log' is optional

## Exercise 3: Combining conditions

Mask the broken pixels with zero counts near the beam stop (center).

- Note that there are pixels at larger scattering angles (larger x) which have real zeros.
  These should not be masked.
- Combine the condition for zero counts with a spatial mask, e.g., based on `x`, to ensure the mask takes only effect close to the direct beam / beam stop.

In [ ]:
# This would mask too much, what needs to be added?
counts.data == 0.0 * sc.Unit('counts')

### Solution

In [ ]:
broken = (counts.data == 0.0 * sc.Unit('counts')) & (sc.abs(x) < 0.1 * sc.Unit('m'))
data.masks['bad-pixels'] = broken
scn.instrument_view(sc.sum(data, 'tof'))

## Exercise 4: More spatial masking

Pick one (or more, if desired):

- Mask a "circle" (in $x$-$y$ plane, i.e., a cylinder aligned with $\hat z$)
- Mask a ring based on $x$ and $y$
- Mask a scattering-angle ($\theta$) range.
  Hint: The scattering angle can be computed as `theta = 0.5 * scn.two_theta(data)`
- Mask a wedge.
  Hint: `phi = sc.atan2(y=y,x=x)`
  
### Solution

In [ ]:
pos = data.coords['position']
x = pos.fields.x
y = pos.fields.y
z = pos.fields.z

# could use offsets x0 and y0 to mask away from z axis
r = sc.sqrt(x * x + y * y)
data.masks['circle'] = r < 0.09 * sc.units.m

data.masks['ring'] = (0.14 * sc.units.m < r) & (r < 0.19 * sc.units.m)

theta = 0.5 * scn.two_theta(data)
data.masks['theta'] = (0.03 * sc.units.rad < theta) & (theta < 0.04 * sc.units.rad)

# sc.to_unit is optional, but useful if we prefer degrees rather than radians
phi = sc.to_unit(sc.atan2(y=y, x=x), unit='deg')
data.masks['wedge'] = (10.0 * sc.units.deg < phi) & (phi < 20.0 * sc.units.deg)

scn.instrument_view(sc.sum(data, 'tof'), norm='log')

## Masks in (grouped) reduction operations

Finally, let us group according to scattering angle and sum spectra.
Questions:

- Can you see the effect of the circle/ring/theta-range that you masked above?
- Why is the prompt-pulse mask preserved, but not the other masks?

In [ ]:
theta_edges = sc.array(dims=['theta'], unit='rad', values=np.linspace(0, 0.1, num=100))
data.coords['theta'] = 0.5 * scn.two_theta(data)
data.groupby(group='theta', bins=theta_edges).sum('spectrum').plot()

### Solution

 - The prompt-pulse mask is preserved since we did not sum over time-of-flight.
 - Masked pixels (spectra) cannot be preserved since we sum over spectra, and the sum simply skips the masked spectra.